# Interfacing DHT-11/ DHT-22 Sensor with Raspberry Pi

Objective: To learn how to interface DHT-11/DHT-22 Sensor with Raspberry pi. Later we shall add SQLITE Database and try to upload the code on Cloud.

In [1]:
%%html
<img src="Photo/3.png" , width=600, height=300>

In [2]:
%%html
<img src="Photo/4.png" , width=600, height=300>

# Please follow complete Steps on how to install the Library on following link 
https://github.com/adafruit/Adafruit_Python_DHT


Installation Steps: -


Python library to read the DHT series of humidity and temperature sensors on a Raspberry Pi or Beaglebone Black.
Designed specifically to work with the Adafruit DHT series sensors ----> https://www.adafruit.com/products/385
Currently the library is tested with Python 2.6, 2.7, 3.3 and 3.4. It should work with Python greater than 3.4, too
Dependencies

For all platforms (Raspberry Pi and Beaglebone Black) make sure your system is able to compile and download Python extensions with pip:

On Raspbian or Beaglebone Black's Debian/Ubuntu image you can ensure your system is ready by running one or two of the following sets of commands

Python 2:
sudo apt-get update
sudo apt-get install python-pip
sudo python -m pip install --upgrade pip setuptools wheel


Python 3:
sudo apt-get update
sudo apt-get install python3-pip
sudo python3 -m pip install --upgrade pip setuptools wheel


Install with pip
Use pip to install from PyPI.


Python 2:
sudo pip install Adafruit_DHT


Python 3:
sudo pip3 install Adafruit_DHT


Compile and install from the repository
First download the library source code from the GitHub releases page, unzipping the archive, and execute:
Python 2:
cd Adafruit_Python_DHT
sudo python setup.py install


Python 3:
cd Adafruit_Python_DHT
sudo python3 setup.py install


You may also git clone the repository if you want to test an unreleased version:
git clone https://github.com/adafruit/Adafruit_Python_DHT.git


# Task 1: Read the sensor.

In [ ]:
mport Adafruit_DHT

# Example using a Raspberry Pi with DHT sensor
# connected to GPIO23.

pin = 4
sensor = Adafruit_DHT.DHT22

humidity, temperature = Adafruit_DHT.read_retry(sensor, pin)

if humidity is not None and temperature is not None:
    print('Temp={0:0.1f}*C  Humidity={1:0.1f}%'.format(temperature, humidity))
else:
    print('Failed to get reading. Try again!')



Run the above Code and if you get the Temperature and humidity congregations and let’s move ahead. Let’s try to read the sensor value continuously we just need to add a while loop


# Task 2: Read the sensor value continuously.

In [ ]:
import Adafruit_DHT

# Example using a Raspberry Pi with DHT sensor
# connected to GPIO23.

pin = 4
sensor = Adafruit_DHT.DHT22

while 1:

    humidity, temperature = Adafruit_DHT.read_retry(sensor, pin)

    if humidity is not None and temperature is not None:
        print('Temp={0:0.1f}*C  Humidity={1:0.1f}%'.format(temperature, humidity))
        if humidity >34:
            print("Alert ")
    else:
        print('Failed to get reading. Try again!')
      


# Task 3: Try to create a UI where you can display Temperature and Humidity continuously 



In [ ]:
import tkinter as tk
import numpy as np
import random
import time
import datetime
import threading
import Adafruit_DHT

pin = 4
sensor = Adafruit_DHT.DHT22

def tick():

    time2=time.strftime('%H:%M:%S')
    clock.config(text=time2)
    clock.after(200,tick)


def get_data():

    threading.Timer(5, get_data).start()

    humidity, temperature = Adafruit_DHT.read_retry(sensor, pin)

    if humidity is not None and temperature is not None:
        print('Temp={0:0.1f}*C  Humidity={1:0.1f}%'.format(temperature, humidity))
        l_display.config(text = temperature)
    else:
        print('Failed to get reading. Try again!')


    return temperature



mainwindow = tk.Tk()
mainwindow.geometry('640x340')
mainwindow.title("Sensor Data Live Feed ")

clock=tk.Label(mainwindow,font=("Arial",30), bg='green',fg="white")
clock.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

l_m=tk.Label(mainwindow,text="Sensor Data ",font=("Arial",30),fg="Black")
l_m.grid(row=0,column=1, padx=10, pady=10, sticky="nsew")

l_t=tk.Label(mainwindow, text="Temperature C",font=("Arial",25))
l_t.grid(row=1,column=0, padx=10, pady=10, sticky="nsew")

l_display=tk.Label(mainwindow,font=("Arial",25),fg="red")
l_display.grid(row=1,column=1, padx=10, pady=10, sticky="nsew")


tick()
get_data()

mainwindow.mainloop()


# Task 4: Try to Log the value on Database 

In [ ]:
import Adafruit_DHT
import sqlite3
import threading
import datetime



def get_time():
    my=datetime.datetime.now()
    data_time = '{}:{}:{}'.format(my.hour,my.minute,my.second)
    data_date='{}/{}/{}'.format(my.day,my.month,my.year)
    return  data_date,data_time


def my_database(x, y,my_time, my_date):

    """

    :return: Nothing
    """

    # define the connection
    conn = sqlite3.connect("xyz.db")

    #define the cursor
    cursor = conn.cursor()

    # create Tables
    cursor.execute("""
     CREATE TABLE IF NOT EXISTS my_table (id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
     temperature  TEXT, 
     humidity TEXT,
     m_date TEXT,
     m_time TEXT)""")

    # Add data to database
    cursor.execute("""
    INSERT INTO my_table (temperature,humidity,m_date,m_time) VALUES (?, ?, ?, ?)""",(x, y,my_time, my_date))

    # perform 3C
    #COMMIT, CLOSE CLOSE

    conn.commit()
    cursor.close()
    conn.close()


def main():
    threading.Timer(10, main).start()

    pin = 4
    sensor = Adafruit_DHT.DHT22

    humidity, temperature = Adafruit_DHT.read_retry(sensor, pin)

    if humidity is not None and temperature is not None:

        print('Temp={0:0.1f}*C  Humidity={1:0.1f}%'.format(temperature, humidity))

        data_date,data_time = get_time()
        print('Date',data_date)



        my_database(temperature, humidity,data_time,data_date)


        print("data was written on database T{} H{}".format(temperature,humidity))

    else:
        print('Failed to get reading. Try again!')


if __name__ == '__main__':
    while 1:
        main()
